In [1]:
import pandas as pd

## Reading in dbNSFP data

In [2]:
db = pd.read_csv('../data/dbNSFP3.2a_gprot_simple_aa_uniq_w_header.txt',
                 sep='\t',
                 index_col=False)

/Users/d/work/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0,4,7,8,9,10,11,12,13,14,15,16,17,20,21,22,24,25,26,27,28,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(db.shape)

(6178371, 34)


In [4]:
software = db[db.columns[db.columns.str.contains('rank')]].columns.to_list()

In [5]:
db['aa_change'] = db['aaref'] + '=>' + db['aaalt']

In [6]:
db = db[['aapos','aa_change','Uniprot_acc_Polyphen2','clinvar_clnsig'] + software]
db = db.drop_duplicates(subset=['aapos','aa_change','Uniprot_acc_Polyphen2'])
db.head()

,aapos,aa_change,Uniprot_acc_Polyphen2,clinvar_clnsig,SIFT_converted_rankscore,Polyphen2_HDIV_rankscore,Polyphen2_HVAR_rankscore,LRT_converted_rankscore,MutationTaster_converted_rankscore,MutationAssessor_score_rankscore,...,integrated_fitCons_score_rankscore,GM12878_fitCons_score_rankscore,H1-hESC_fitCons_score_rankscore,HUVEC_fitCons_score_rankscore,GERP++_RS_rankscore,phyloP100way_vertebrate_rankscore,phyloP20way_mammalian_rankscore,phastCons100way_vertebrate_rankscore,phastCons20way_mammalian_rankscore,SiPhy_29way_logOdds_rankscore
0,201;129,E=>D,Q9Y5U5,.,0.16119,0.13521,0.11238,.,0.81033,.,...,0.37169,0.55475,0.27661,0.37612,0.21098,0.11537,0.15898,0.06329,0.16931,0.22632
2,201;129,E=>V,Q9Y5U5,.,0.3164,0.51906,0.47406,.,0.81033,.,...,0.37169,0.55475,0.27661,0.37612,0.19134,0.09480,0.21895,0.06329,0.16870,0.16314
3,201;129,E=>G,Q9Y5U5,.,0.17853,0.06737,0.07978,.,0.81033,.,...,0.37169,0.55475,0.27661,0.37612,0.19134,0.09480,0.21895,0.06329,0.16870,0.16314
4,201;129,E=>A,Q9Y5U5,.,0.17269,0.36774,0.30254,.,0.81033,.,...,0.37169,0.55475,0.27661,0.37612,0.19134,0.09480,0.21895,0.06329,0.16870,0.16314
5,201;129,E=>Q,Q9Y5U5,.,0.16226,0.51906,0.47406,.,0.81033,0.37555,...,0.28343,0.40457,0.27661,0.20234,0.37198,0.26938,0.48983,0.27262,0.19595,0.57778


In [7]:
print(db.shape)

(5326058, 31)


## Create simple table for just proteins with glycosylation sites with distance annotation

In [8]:
p_d = pd.read_csv('../data/protein_aa_distance.txt',
                       sep='\t',
                       header=None,index_col=0)

In [9]:
p_d.columns = ['proteins','aa','distance_to_feature']

## Going through the clinvar database and matching amino acids


In [10]:
pd_dict = {'aa':[],
           'proteins':[],
           'clinvar_path':[],
           'aa_change':[],
           'clinvar_index':[]}
           
for s in software:
    pd_dict[s] = []

for i in db.index:
    one_var = db.loc[i]
    aa_list = one_var['aapos']
    if not isinstance(aa_list,int):
        aa_list = one_var['aapos'].split(';')
    else:
        aa_list = [one_var['aapos']]
    protein_list = one_var['Uniprot_acc_Polyphen2'].split(';')
    path = one_var['clinvar_clnsig']
    aa_change = one_var['aa_change']
    for p in protein_list:
        for aa in aa_list:
            pd_dict['aa'].append(aa)
            pd_dict['proteins'].append(p)
            pd_dict['clinvar_path'].append(path)
            pd_dict['aa_change'].append(aa_change)
            pd_dict['clinvar_index'].append(i)
            for s in software:
                pd_dict[s].append(one_var[s])

In [11]:
flat_db = pd.DataFrame.from_dict(pd_dict)
flat_db['aa'] = flat_db['aa'].astype(int)

In [12]:
print(flat_db.shape)
print('Dropping duplicates')
flat_db =  flat_db.drop_duplicates(subset=['aa','proteins','aa_change'])
print(flat_db.shape)

(32789539, 32)
Dropping duplicates
(21791618, 32)


In [13]:
p_d.head()

,proteins,aa,distance_to_feature
0,,,
262180,P16422,74,0.0
906903,Q9UJW2,175,0.0
3197828,Q16280,379,0.0
2176002,Q8NFQ6,113,0.0
2551366,O15455,70,0.0


In [14]:
flat_db.head()

,aa,proteins,clinvar_path,aa_change,clinvar_index,SIFT_converted_rankscore,Polyphen2_HDIV_rankscore,Polyphen2_HVAR_rankscore,LRT_converted_rankscore,MutationTaster_converted_rankscore,...,integrated_fitCons_score_rankscore,GM12878_fitCons_score_rankscore,H1-hESC_fitCons_score_rankscore,HUVEC_fitCons_score_rankscore,GERP++_RS_rankscore,phyloP100way_vertebrate_rankscore,phyloP20way_mammalian_rankscore,phastCons100way_vertebrate_rankscore,phastCons20way_mammalian_rankscore,SiPhy_29way_logOdds_rankscore
0,201,Q9Y5U5,.,E=>D,0,0.16119,0.13521,0.11238,.,0.81033,...,0.37169,0.55475,0.27661,0.37612,0.21098,0.11537,0.15898,0.06329,0.16931,0.22632
1,129,Q9Y5U5,.,E=>D,0,0.16119,0.13521,0.11238,.,0.81033,...,0.37169,0.55475,0.27661,0.37612,0.21098,0.11537,0.15898,0.06329,0.16931,0.22632
2,201,Q9Y5U5,.,E=>V,2,0.3164,0.51906,0.47406,.,0.81033,...,0.37169,0.55475,0.27661,0.37612,0.19134,0.09480,0.21895,0.06329,0.16870,0.16314
3,129,Q9Y5U5,.,E=>V,2,0.3164,0.51906,0.47406,.,0.81033,...,0.37169,0.55475,0.27661,0.37612,0.19134,0.09480,0.21895,0.06329,0.16870,0.16314
4,201,Q9Y5U5,.,E=>G,3,0.17853,0.06737,0.07978,.,0.81033,...,0.37169,0.55475,0.27661,0.37612,0.19134,0.09480,0.21895,0.06329,0.16870,0.16314


In [15]:
merged = flat_db.merge(p_d,on=['aa','proteins'],how='outer')

In [16]:
print(merged.shape)

(21841733, 33)


# Dropping any variants that are not in glycosylated proteins

In [17]:
g_proteins = list(p_d['proteins'].unique())

In [18]:
merged = merged[merged['proteins'].isin(g_proteins)]
print(merged.shape)

(9161749, 33)


# Dealing with variants that do not have a distance to glycosylation site
Need to be minful here: can either filter these, add psudo distance or keep as background


In [19]:
## Dropping NAs
print(merged.shape)
merged = merged[~merged['distance_to_feature'].isna()]
print(merged.shape)

(9161749, 33)
(7860706, 33)


## Filtering sites where distance to feature == 0, which is presumably the glycosylation site

In [20]:
merged = merged[merged['distance_to_feature'] != 0]
merged.shape

(7785647, 33)

## Dropping duplicates that have same aa, aapos and , keeping closest to glycosite

In [21]:
merged = merged.sort_values(by='distance_to_feature',ascending=True).drop_duplicates(subset=['aa','proteins','aa_change'],keep='first')

In [22]:
print(merged.shape)

(7785647, 33)


## Dropping variants that don't have at least 4 scores

In [23]:
merged = merged.dropna(subset=software,thresh=3)
print(merged.shape)

(7735889, 33)


In [24]:
merged.shape

(7735889, 33)

## Adding in glycoimpact score

In [25]:
gi = pd.read_csv('../data/glyco_protein_impact_within_6.csv')

In [26]:
gi['aa_change'] = gi['substitution']

In [27]:
merged = merged.merge(gi,on='aa_change',how='outer').dropna(subset=['aa'])

In [28]:
merged.to_csv('../data/glycoproteins_distances_with_pathsoftware.txt',sep='\t')